<a href="https://colab.research.google.com/github/kimjihun-gif/Echinococcus_multilocularis_Phylogenetic_analysis/blob/main/phylo_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# NOTE: These are required only in Colab or fresh environments
!pip install biopython
!apt-get install -y clustalo

# Purpose: This script was written to retrieve Echinococcus multilocularis mitochondrial sequences (cox1 gene),
# extract cox1 regions, align sequences, compute genetic distances, and construct a bootstrap-supported phylogenetic tree.

from Bio import Entrez, SeqIO
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.Align.Applications import ClustalOmegaCommandline
from Bio import AlignIO, Phylo
from Bio.Phylo.TreeConstruction import DistanceCalculator, DistanceTreeConstructor
from Bio.Phylo.Consensus import majority_consensus
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# The email below is required for NCBI Entrez API usage.
# Replace with your own institutional email if submitting to a journal or using extensively.
Entrez.email = "example@example.com"

accessions = [
    "AB780998.1",
    "LC720770.1",
    "LC720726",
    "LC720728",
    "LC720730",
    "LC720731",
    "LC720734"
]

records = []
for acc in accessions:
    handle = Entrez.efetch(db="nucleotide", id=acc, rettype="gb", retmode="text")
    record = SeqIO.read(handle, "genbank")
    handle.close()

    cox1_seq = None
    for feature in record.features:
        if feature.type == "CDS":
            gene_name = feature.qualifiers.get("gene", [""])[0].lower()
            product_name = feature.qualifiers.get("product", [""])[0].lower()
            if "cox1" in gene_name or "cox1" in product_name or "cytochrome oxidase subunit 1" in product_name:
                location = feature.location
                cox1_seq = location.extract(record.seq)
                break

    if cox1_seq:
        seq_record = SeqIO.SeqRecord(cox1_seq, id=acc, description="cox1")
        records.append(seq_record)

# Write only cox1 regions
SeqIO.write(records, "cox1_only.fasta", "fasta")

# Alignment with Clustal Omega
clustalomega_cline = ClustalOmegaCommandline(infile="cox1_only.fasta", outfile="aligned.fasta", verbose=True, auto=True)
os.system(str(clustalomega_cline))

# Load aligned sequences
alignment = AlignIO.read("aligned.fasta", "fasta")
calculator = DistanceCalculator("identity")
dm = calculator.get_distance(alignment)

# Save distance matrix
distance_df = pd.DataFrame(dm.matrix, index=dm.names, columns=dm.names)
distance_df.to_csv("cox1_distance_matrix.csv")

# Build phylogenetic tree
constructor = DistanceTreeConstructor()

def bootstrap_alignment(alignment, n_replicates=100):
    length = alignment.get_alignment_length()
    alignments = []
    for _ in range(n_replicates):
        indices = np.random.randint(0, length, length)
        bootstrap_records = []
        for record in alignment:
            bootstrap_seq = "".join(record.seq[i] for i in indices)
            bootstrap_record = record[:]
            bootstrap_record.seq = bootstrap_seq
            bootstrap_records.append(bootstrap_record)
        alignments.append(AlignIO.MultipleSeqAlignment(bootstrap_records))
    return alignments

# Bootstrap and consensus tree
bootstrap_alignments = bootstrap_alignment(alignment, n_replicates=100)
bootstrap_trees = [constructor.nj(calculator.get_distance(align)) for align in bootstrap_alignments]
consensus_tree = majority_consensus(bootstrap_trees, cutoff=0.5)

from Bio.Phylo import draw
from io import StringIO
# Draw tree and add bootstrap values at better positions
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(1, 1, 1)
Phylo.draw(consensus_tree, axes=ax, do_show=False)

# Manually plot confidence values with minimal overlap
x_offset = 0.0002  # slight shift to the right
y_offset = 0.3     # slight shift upward to avoid labels

for i, clade in enumerate(consensus_tree.find_clades(order="level")):
    if clade.confidence and clade.branch_length:
        x = clade.branch_length + x_offset
        y = i + y_offset
        ax.text(x, y, f"{clade.confidence:.1f}", color="red", fontsize=9)

plt.tight_layout()
plt.show()
